# Standardizing Financial Data by Recent Return and Volatility

An interesting idea is to standardize the returns of financial markets by both the recent returns and recent volatility. Standardizing by recent returns will return lower values in strong uptrends and higher values in strong downtrends. Standardizing by recent volatility will return lower values in volatile markets and higher values in low volatility markets.
We see the results of a simple test below.

It is very possible to data mine around this by optimizing across the period parameter of the standardization as well as the holding period.

Standardizing by volatility is very interesting as it gives perspective on how large a market move is in terms of recent ranges of the market. This can be useful in putting market data in perspective of the recent market structure.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
SPX = pd.read_csv("C:/Users/Sam/Desktop/Cleaned and Formatted Data/ES Rev Adj.csv", delimiter = ",", index_col = "Date", parse_dates = True)

In [3]:
def standardize(series, period, series2):
    return ((series - series.rolling(period).mean())/(series2.rolling(period).std()))

# Create a Range and Return column, alongside a forward return of 1, 5, and 10 days for each day

SPX['Range'] = SPX['High'] - SPX['Low']
SPX['Return'] = SPX['Close']/SPX['Close'].shift(1)

SPX['Forward Return 1D'] = SPX['Close'].shift(-1) - SPX['Close']
SPX['Forward Return 5D'] = SPX['Close'].shift(-5) - SPX['Close']
SPX['Forward Return 10D'] = SPX['Close'].shift(-10) - SPX['Close']

Below we find the code to standardize the daily returns.

In [4]:
SPX['Return by Return'] = standardize(SPX['Return'], 20, SPX['Return'])
SPX['Return by Vol'] = standardize(SPX['Return'], 20, SPX['High']/SPX['Low'])

In [5]:
def Bucket_Returns(days, column, start, end):
    df = SPX['Forward Return '+str(days)+'D'][((SPX[column] >= start) & (SPX[column] <= end))]
    print(str(days) + ' Day Returns with '+column+' from '+str(start)+' to '+str(end)+' are:' +  
          str(df.mean()) + ' with ' + str(df.shape[0]) + ' records')

s2 = list(np.arange(-6, 6, 1))
e2 = list(np.arange(-5, 7, 1))

In [6]:
for i in range(len(s2)):
    Bucket_Returns(5, 'Return by Return', round(s2[i]), round(e2[i]))

5 Day Returns with Return by Return from -6 to -5 are:nan with 0 records
5 Day Returns with Return by Return from -5 to -4 are:nan with 0 records
5 Day Returns with Return by Return from -4 to -3 are:31.392857142857142 with 14 records
5 Day Returns with Return by Return from -3 to -2 are:1.4613970588235294 with 136 records
5 Day Returns with Return by Return from -2 to -1 are:4.016280752532562 with 691 records
5 Day Returns with Return by Return from -1 to 0 are:2.4998439937597503 with 1926 records
5 Day Returns with Return by Return from 0 to 1 are:0.2703888596952182 with 1904 records
5 Day Returns with Return by Return from 1 to 2 are:0.3177004538577912 with 662 records
5 Day Returns with Return by Return from 2 to 3 are:-0.765625 with 96 records
5 Day Returns with Return by Return from 3 to 4 are:10.583333333333334 with 3 records
5 Day Returns with Return by Return from 4 to 5 are:nan with 0 records
5 Day Returns with Return by Return from 5 to 6 are:nan with 0 records


Although rare, when returns are more than 3 standard deviations below the average of the past 20 days, this seems an incredible short term buying opportunity.

In [7]:
for i in range(len(s2)):
    Bucket_Returns(5, 'Return by Vol', round(s2[i]), round(e2[i]))

5 Day Returns with Return by Vol from -6 to -5 are:17.113636363636363 with 11 records
5 Day Returns with Return by Vol from -5 to -4 are:3.212121212121212 with 66 records
5 Day Returns with Return by Vol from -4 to -3 are:5.244285714285715 with 175 records
5 Day Returns with Return by Vol from -3 to -2 are:3.1392405063291138 with 395 records
5 Day Returns with Return by Vol from -2 to -1 are:3.8092669432918393 with 725 records
5 Day Returns with Return by Vol from -1 to 0 are:2.0865136298421807 with 1395 records
5 Day Returns with Return by Vol from 0 to 1 are:0.1918474195961107 with 1337 records
5 Day Returns with Return by Vol from 1 to 2 are:0.3458559782608695 with 737 records
5 Day Returns with Return by Vol from 2 to 3 are:-0.7622739018087855 with 388 records
5 Day Returns with Return by Vol from 3 to 4 are:3.875 with 146 records
5 Day Returns with Return by Vol from 4 to 5 are:-6.808139534883721 with 43 records
5 Day Returns with Return by Vol from 5 to 6 are:12.48076923076923 wi

We see that when the market drops by more than 1 standard deviation of the range over the past 20 days it is usually a good short term buy. Also interesting is that there is no clear edge buying the market when it is up from this methodology.